In [2]:
import numpy as np
import pandas as pd
import time
import sys
sys.path.append(r"C:\Program Files\IBM\ILOG\CPLEX_Studio2211\cplex\python\3.9\x64_win64")
import cplex
!pip install docplex
import docplex.mp.model as mip

You should consider upgrading via the 'C:\Users\mukun\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [48]:
df = pd.read_csv(r"C:\Users\mukun\Downloads\1year_seed1_instance_sampling.csv")
#df = pd.read_csv(r"C:\Users\mukun\Downloads\instance_seed562.csv")
df

,Week,Day,Weekday,Event Type,Next Event Type,Ticket Price,Attendance,Overhead,Setup Costs,Cleanup Costs,Special Event
0,1,1,1,0,0,251.416136,20041.0,62824.41,4345.024729,6373.633657,0
1,1,1,1,1,0,46.457329,17766.0,62824.41,4961.612669,7775.984667,0
2,1,1,1,2,0,31.831421,22090.0,62824.41,4983.697417,9999.905085,0
3,1,1,1,3,0,51.922898,20680.0,62824.41,6167.999266,12094.826171,0
4,1,1,1,4,0,120.343136,16450.0,62824.41,3541.341350,8211.990881,0
...,...,...,...,...,...,...,...,...,...,...,...
1820,53,365,1,0,0,163.056117,17927.0,62824.41,3826.479779,6347.969696,0
1821,53,365,1,1,0,103.845795,16667.0,62824.41,4460.178258,7626.949575,0
1822,53,365,1,2,0,116.488955,20445.0,62824.41,4706.807298,10008.365782,0
1823,53,365,1,3,0,46.457329,19505.0,62824.41,6754.071902,10023.593001,0


In [49]:
df = df.drop(['Weekday', 'Next Event Type'], axis = 1)
df['Day'] = df['Day'] -1
df['Week'] = df['Week'] -1
df2 = df.groupby(['Day', 'Event Type']).mean()
n = df2.Attendance
s = df2['Ticket Price'].abs()
SC = df2['Setup Costs']
CC = df2['Cleanup Costs']
OC = df2.Overhead


In [50]:
special_events = df2['Special Event']
special_events = special_events[special_events == 1]
special_events
special_event_days = []


for i in range(len(special_events)):
  special_event_days.append(special_events.index[i][0])
special_event_days
special_event_days = special_event_days #for now
special_event_days

[8, 25, 33, 81, 94, 113, 137, 180, 209, 211, 273, 275, 295, 312, 315, 338]

In [51]:
events = [0,1,2,3,4]
sports_events = [0,1]
days = []
for i in range(364):
  days.append(i)
in_season = []
for i in range(231):
  in_season.append(i)
off_season = []
for i in range(231,364):
  off_season.append(i)

weeks = []
for i in range(52):
  weeks.append(i)

weekset = []
weekday = []
weeknumber = 0
for i in weeks:
  for j in range(7):
    weekday.append(weeknumber)
    weeknumber += 1
  weekset.append(weekday)
  weekday = []

In [52]:
df2

Week  Ticket Price  Attendance  Overhead  Setup Costs   
Day Event Type                                                          
0   0            0.0    251.416136     20041.0  62824.41  4345.024729  \
    1            0.0     46.457329     17766.0  62824.41  4961.612669   
    2            0.0     31.831421     22090.0  62824.41  4983.697417   
    3            0.0     51.922898     20680.0  62824.41  6167.999266   
    4            0.0    120.343136     16450.0  62824.41  3541.341350   
...              ...           ...         ...       ...          ...   
364 0           52.0    163.056117     17927.0  62824.41  3826.479779   
    1           52.0    103.845795     16667.0  62824.41  4460.178258   
    2           52.0    116.488955     20445.0  62824.41  4706.807298   
    3           52.0     46.457329     19505.0  62824.41  6754.071902   
    4           52.0    118.753012     16685.0  62824.41  3562.150263   

                Cleanup Costs  Special Event  
Day Event Type                                
0   0             6373.633657            0.0  
    1             7775.984667            0.0  
    2             9999.905085            0.0  
    3            12094.826171            0.0  
    4             8211.990881            0.0  
...                       ...            ...  
364 0             6347.969696            0.0  
    1             7626.949575            0.0  
    2            10008.365782            0.0  
    3            10023.593001            0.0  
    4             9710.185483            0.0  

[1825 rows x 7 columns]

In [ ]:
#@title MIP Model Completed

#Model
mip_md1 = mip.Model("MIP")
#Variable creation
x = mip_md1.integer_var_dict(keys = ((i,j) for i in days for j in events), lb = 0, ub = 1, name = 'x')
z = mip_md1.integer_var_dict(keys = ((m,n) for m in in_season for n in sports_events), lb = 0, ub = 1, name = 'z')

mip_md1.parameters.mip.tolerances.mipgap.set(1e-6) 
mip_md1.context.solver.log_output = True

#Constraints
#At most, only one event can occur per day, although the venue can also have no events scheduled for a day as well
for i in days:
  mip_md1.add_constraint(mip_md1.sum(x[i,j] for j in events) <= 1) #yes

#Venue’s home basketball team must play exactly 41 games at the venue. A typical NBA team plays 82 games per season, of which it is expected that half will be at the home venue
mip_md1.add_constraint(mip_md1.sum(x[t,0] for t in in_season) == 41) #yes

#Similarly, the venue’s home hockey team must play exactly 41 games at the venue, as a typical NHL team plays 82 games per season
mip_md1.add_constraint(mip_md1.sum(x[t,1] for t in in_season) == 41) #yes

#There must be no games scheduled in the sports offseason, that is, between days 212-365.
for i in sports_events:
  mip_md1.add_constraint(mip_md1.sum(x[t,i] for t in off_season) == 0) #yes

#There cannot be three days in a row where the same sporting event is played.
for i in sports_events:
  for t in range(len(in_season)-2):
    mip_md1.add_constraint(mip_md1.sum(x[k,i] for k in range(t,t+3)) <= 2) #yes

#The total number of days where the venue is in use must be less than 300.
mip_md1.add_constraint(mip_md1.sum(x[i,j] for i in days for j in events) <= 300) #yes

#The total number of days where the venue is in use must be at least 100.
mip_md1.add_constraint(mip_md1.sum(x[i,j] for i in days for j in events) >= 100) #yes

#There can be up to 4 events scheduled within a 7 day period, but no more.
for k in weekset:
  mip_md1.add_constraint(mip_md1.sum(x[w,i] for w in k for i in events) <= 4) #yes

#There cannot be more than 3 sporting events scheduled in a 7 day period.
for k in weekset:
  mip_md1.add_constraint(mip_md1.sum(x[w,i] for w in k for i in sports_events) <= 3) #yes
 
#At least one concert must be scheduled in a given 7-day period.
for k in weekset:
  mip_md1.add_constraint(mip_md1.sum(x[w,2] for w in k) >= 1) #yes

#There have to be at least 15 exhibitions/conventions scheduled a year.
mip_md1.add_constraint(mip_md1.sum(x[i,3] for i in days) >= 15) #yes



# There must be 1 day of lag after a concert to account for takedown/cleanup time.
for t in range(len(days) - 1):
    for i in events:
        mip_md1.add_constraint(x[t, 2] + x[t + 1, i] <= 1) #yes

# There must be 1 day of lag before a concert to account for setup time.
for t in range(1, len(days)):
    for i in events:
        mip_md1.add_constraint(x[t - 1, i] + x[t, 2] <= 1) #yes

#Back to back basketball games, and back to back hockey games may only occur up to 8 times for each sport, but no more.
for i in sports_events:
  for j in range(len(in_season) - 1):

    mip_md1.add_constraint(z[j,i] <= x[j,i])
    mip_md1.add_constraint(z[j,i] <= x[j+1, i])
    mip_md1.add_constraint(z[j,i] >= x[j,i] + x[j+1, i] - 1)

for i in sports_events:
  mip_md1.add_constraint(mip_md1.sum(z[t,i] for t in range(len(in_season)-1)) <= 8) #yes

for t in special_event_days:
  mip_md1.add_constraint(x[t,4] == 1)

mip_md1.maximize(mip_md1.sum(s.loc[(t,i)] * n.loc[(t,i)] * x[t,i] - SC.loc[(t,i)]*x[t,i] - CC.loc[(t,i)]*x[t,i] - OC.loc[(t,i)]*x[t,i] for t in days for i in events))


#initial_vars = mip_md1.number_of_variables
#print(initial_vars)
#num_constraints_before_presolve = mip_md1.number_of_constraints
# Solve the model
mip_md1.parameters.mip.strategy.search = 1
soln = mip_md1.solve(log_output=False)
with open("solver_log.txt", "w") as log_file:
    mip_md1.solve(log_output=log_file)


with open("solver_log_365_instance_1.txt", "w") as log_file:

    mip_md1.solve(log_output=log_file)

    if soln is not None:
        log_file.write(f"Objective Value: {mip_md1.objective_value}\n")
        if soln is not None:
            original_stdout = sys.stdout
            sys.stdout = log_file  
            mip_md1.print_solution()  
            sys.stdout = original_stdout 
        else:
            print("* Model is infeasible")

if soln is not None:
  total_ones_in_x = sum(value for (i, j), value in mip_md1.solution.get_value_dict(x).items() if value == 1)
  print({total_ones_in_x})
  mip_md1.print_solution()

else:
  print("* model is infeasible")

stats = mip_md1.get_statistics()



Total number of 1s in x variables: 203.0
objective: 784711512.420
status: OPTIMAL_SOLUTION(2)
  x_0_0=1
  x_3_0=1
  x_4_0=1
  x_6_2=1
  x_8_4=1
  x_9_0=1
  x_11_0=1
  x_13_2=1
  x_15_0=1
  x_16_1=1
  x_17_1=1
  x_19_2=1
  x_21_2=1
  x_24_0=1
  x_25_4=1
  x_27_0=1
  x_28_0=1
  x_30_2=1
  x_32_0=1
  x_33_4=1
  x_35_1=1
  x_36_0=1
  x_38_1=1
  x_40_2=1
  x_42_0=1
  x_45_0=1
  x_46_1=1
  x_48_2=1
  x_50_1=1
  x_51_1=1
  x_53_2=1
  x_55_1=1
  x_56_1=1
  x_57_4=1
  x_59_2=1
  x_61_2=1
  x_64_1=1
  x_65_0=1
  x_66_0=1
  x_68_2=1
  x_71_0=1
  x_73_2=1
  x_75_0=1
  x_76_0=1
  x_78_2=1
  x_80_1=1
  x_81_4=1
  x_82_0=1
  x_84_0=1
  x_87_0=1
  x_88_1=1
  x_90_2=1
  x_92_1=1
  x_92_2=0
  x_93_1=1
  x_94_4=1
  x_96_1=-0
  x_96_2=1
  x_98_1=1
  x_101_2=1
  x_103_1=1
  x_104_0=1
  x_105_1=1
  x_108_2=1
  x_110_1=1
  x_111_0=1
  x_113_4=1
  x_115_0=1
  x_116_0=1
  x_118_2=1
  x_120_1=1
  x_122_2=1
  x_124_0=1
  x_125_4=1
  x_126_1=1
  x_128_2=1
  x_130_2=1
  x_132_1=1
  x_134_2=1
  x_136_1=1
  x_137_4=

In [14]:
months = ['Oct','Nov','Dec','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sept']
months_of_year = dict.fromkeys(months, [])
months_of_year_range = [(0,31), (31,61),(61,92),(92,122),(122,153),(153,181),(181,212),(212,242),(242,273),(273,304),(304,335),(335,364)]
zipped = zip(months, months_of_year_range)
month_range = dict.fromkeys(months,[])
for i, j in zipped:
  month_range[i] = j
month_range

{'Oct': (0, 31),
 'Nov': (31, 61),
 'Dec': (61, 92),
 'Jan': (92, 122),
 'Feb': (122, 153),
 'Mar': (153, 181),
 'Apr': (181, 212),
 'May': (212, 242),
 'Jun': (242, 273),
 'Jul': (273, 304),
 'Aug': (304, 335),
 'Sept': (335, 364)}

In [15]:
for mth, days in month_range.items():
  month_list = []
  for i in x:
    if i[0] in range(days[0],days[1]):
      value = mip_md1.solution.get_value(x[i])
      if value == 1:
        month_list.append(i)
  months_of_year[mth] = month_list
print(months_of_year)

{'Oct': [(1, 2), (3, 1), (5, 1), (6, 0), (7, 1), (8, 1), (10, 2), (12, 0), (14, 2), (17, 1), (18, 0), (20, 2), (22, 4), (24, 2), (26, 0), (27, 1), (29, 0), (30, 4)], 'Nov': [(32, 4), (34, 2), (37, 2), (39, 0), (40, 0), (41, 1), (42, 4), (44, 2), (46, 1), (47, 1), (49, 2), (51, 2), (53, 0), (54, 0), (56, 0), (57, 1), (58, 1), (60, 2)], 'Dec': [(63, 2), (66, 2), (68, 0), (69, 0), (71, 0), (72, 4), (74, 2), (76, 0), (78, 2), (80, 0), (82, 0), (83, 1), (85, 4), (86, 1), (87, 4), (89, 2), (91, 0)], 'Jan': [(93, 1), (95, 2), (97, 0), (99, 2), (102, 1), (103, 0), (104, 1), (105, 1), (108, 1), (109, 1), (111, 2), (114, 2), (116, 1), (117, 4), (118, 0), (119, 1), (120, 1)], 'Feb': [(122, 2), (124, 4), (126, 2), (129, 1), (130, 0), (131, 0), (133, 2), (135, 1), (137, 1), (139, 2), (141, 2), (143, 1), (145, 0), (146, 0), (147, 1), (148, 1), (149, 0), (151, 2)], 'Mar': [(154, 0), (155, 4), (157, 2), (160, 1), (161, 0), (163, 1), (164, 0), (166, 2), (168, 0), (169, 1), (172, 2), (174, 1), (175, 0),

In [121]:
for i in months_of_year:
  print(len(months_of_year[i]))

18
17
17
17
19
15
19
16
18
18
17
16


In [ ]:
flattened_data = [(month, day, value) for month, days in months_of_year.items() for day, value in days]
df = pd.DataFrame(flattened_data, columns=['Month', 'Day', 'Value'])

df.to_csv('instance_0_scheduele.csv', index=False)

In [ ]:
import os
os.getcwd()

'c:\\Users\\mukun'

In [ ]:
days = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
days_of_week = {}
for day in days:
  days_of_week[day] = []
  
for i in x:
    day_idx = i[0] % 7  
    value = mip_md1.solution.get_value(x[i])  
    if value == 1:
        days_of_week[days[day_idx]].append(i)  

print(days_of_week)

{'Sunday': [(0, 2), (28, 2), (42, 1), (49, 1), (56, 0), (63, 2), (70, 2), (84, 1), (91, 2), (98, 1), (119, 1), (133, 1), (147, 2), (161, 0), (168, 0), (175, 2), (196, 1), (203, 2), (217, 0), (252, 2), (266, 3), (301, 4), (308, 2), (329, 2), (350, 4)], 'Monday': [(1, 2), (8, 1), (36, 0), (57, 1), (113, 1), (120, 1), (127, 2), (141, 4), (155, 2), (169, 1), (183, 0), (190, 1), (211, 0), (218, 0), (225, 2), (232, 4), (239, 4), (288, 2), (309, 4), (316, 2), (330, 2), (344, 2)], 'Tuesday': [(9, 0), (16, 2), (23, 2), (30, 1), (37, 0), (51, 2), (58, 1), (79, 4), (86, 2), (100, 2), (107, 1), (114, 0), (135, 0), (142, 4), (163, 1), (191, 0), (198, 0), (205, 4), (219, 4), (233, 2), (240, 4), (254, 2), (261, 3), (275, 2), (289, 4), (303, 2), (338, 2)], 'Wednesday': [(10, 1), (24, 4), (31, 0), (38, 1), (45, 2), (73, 0), (80, 2), (94, 0), (108, 1), (150, 4), (157, 1), (171, 2), (185, 0), (199, 0), (213, 2), (234, 2), (248, 3), (276, 4), (283, 2), (297, 3), (318, 2), (325, 2), (353, 2), (360, 2)], 'T